In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Import Data
df = pd.read_csv("tp_df.csv")
df = df.drop(["Unnamed: 0"], axis=1)

In [ ]:
# Drop Errors and NAN values
df = df[df['turn_over_rate']<1]
error_firm = ['동원홈푸드','휠라홀딩스','트리','와디즈','키위컴퍼니','줌인터넷','시선인터내셔널','브이티코스메틱','유니슨이테크','씨엠비대전방송','서울비젼','더메인즈','조은시스템']

for i in error_firm:
    df = df[df.company_name != i]

# Rename the columns
df = df.rename(columns={"average_salary":"average_salary(만원)", "total_sale":"total_sale(억원)"})


In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib import rc

# for Mac
rc('font', family='AppleGothic')

plt.rcParams['axes.unicode_minus'] = False

# Get turn_over_rate graphs

plt.title('turn_over_rate')
plt.xticks(rotation = 45)
sns.histplot(df['turn_over_rate'],kde=True)
plt.show()

In [ ]:
adv_list = df.columns[1:7]
adv_col = 3
dadv_list = df.columns[7:-3]
dadv_col = 4
fv_list = df.columns[-3:-1]
fv_col = 1

def graphing_data (col, num):
    plt.figure(figsize=(24,12))
    for i, title in enumerate(col):
        plt.subplot(2,num,i+1)
        plt.title(title+" scatter plot")
        sns.scatterplot(x= df[title].astype(float), y = df['turn_over_rate'])

    # if col[0][-1].isdigit():
    #     if col[0][0] == 'a':
    #         plt.savefig('Advatage_topic_modeling_scatter_plot.png')
    #     if col[0][0] == 'd':
    #         plt.savefig('Disadvatage_topic_modeling_scatter_plot.png')
    # else:
    #     plt.savefig('Financial_variable_scatter_plot.png')
    plt.show()

In [ ]:
graphing_data(adv_list, adv_col)

In [ ]:
graphing_data(dadv_list,dadv_col)

In [ ]:
graphing_data(fv_list,fv_col)

**Machine Learning**

Linear, Ridge, Lasso

In [ ]:
def get_rmse(model):
    pred = model.predict(X_test.astype(float))
    mse = mean_squared_error(y_test.astype(float), pred)
    rmse = np.sqrt(mse)
    print(model.__class__.__name__, ' RMSE: ', np.round(rmse,3))
    return rmse

In [ ]:
def get_mae(model):
    pred = model.predict(X_test.astype(float))
    score = mean_absolute_error(y_test.astype(float),pred)
    print(model.__class__.__name__, " MAE: ", np.round(score,3))

In [ ]:
def get_rmses(models):
    rmses = []
    for model in models:
        rmse = get_rmse(model)
        # rmses.append(rmse)
    # return rmses

In [ ]:
def get_maes(models):
    scores = []
    for model in models:
        score = get_mae(model)
        # scores.append(score)
    # return scores

In [ ]:
# Linear Regression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_error

y_target = df['turn_over_rate']
x_data = df.drop(['company_name','turn_over_rate'], axis = 1, inplace = False)

X_train, X_test, y_train, y_test = train_test_split(x_data, y_target, test_size = 0.1, random_state = 7)

lr_reg = LinearRegression()
lr_reg.fit(X_train, y_train)

ridge_reg = Ridge()
ridge_reg.fit(X_train, y_train)

lasso_reg = Lasso()
lasso_reg.fit(X_train,y_train)

models = [lr_reg, ridge_reg, lasso_reg]
get_rmses(models)
print('\n')
get_maes(models)

In [ ]:
def get_top_bottom_coef(model, n=4):

    coef = pd.Series(model.coef_, index=x_data.columns)

    coef_high = coef.sort_values(ascending=False).head(n)
    coef_low = coef.sort_values(ascending=False).tail(n)

    return coef_high, coef_low

In [ ]:
def visualize_coefficient(models):
    fig, axs = plt.subplots(figsize = (24,10), nrows = 1, ncols= 3)
    fig.tight_layout()

    for i_num, model in enumerate(models):

        coef_high, coef_low = get_top_bottom_coef(model)
        coef_concat = pd.concat([coef_high, coef_low])

        axs[i_num].set_title(model.__class__.__name__+' Coefficients', size=25)
        axs[i_num].tick_params(axis="y", direction="in",pad=-120)
        for label in (axs[i_num].get_xticklabels() + axs[i_num].get_yticklabels()):
            label.set_fontsize(22)
        sns.barplot(x=coef_concat.values, y=coef_concat.index, ax = axs[i_num])

models = [lr_reg, ridge_reg, lasso_reg]
visualize_coefficient(models)

In [ ]:
from sklearn.model_selection import cross_val_score

def get_avg_rmse_cv(models):

    for model in models:

        rmse_list = np.sqrt(-cross_val_score(model, x_data, y_target,
                                             scoring= 'neg_mean_squared_error', cv = 5))
        rmse_avg = np.mean(rmse_list)
        print('\n{0} CV RMSE Value List: {1}'.format(model.__class__.__name__, np.round(rmse_list,3)))
        print('\n{0} CV average RMSE Value List: {1}'.format(model.__class__.__name__, np.round(rmse_avg,3)))

models = [ridge_reg, lasso_reg]
get_avg_rmse_cv(models)

In [ ]:
from sklearn.model_selection import GridSearchCV

def print_best_params(model, params):

    grid_model = GridSearchCV(model, param_grid=params,
                              scoring='neg_mean_squared_error', cv=5)
    grid_model.fit(X_train.astype(float),y_train.astype(float))
    mse = -1 * grid_model.best_score_
    rmse = np.sqrt(-1*grid_model.best_score_)
    print('{0} After 5 CV, best average MSE: {1}, best average RMSE: {2}, best alpha: {3}'.format(model.__class__.__name__,np.round(mse,4),
                                                                           np.round(rmse,4),grid_model.best_params_))
    print('\n')     


def save_best_params(model, params):

    grid_model = GridSearchCV(model, param_grid=params,
                              scoring='neg_mean_squared_error', cv=5)
    grid_model.fit(X_train.astype(float),y_train.astype(float))
    mse = -1 * grid_model.best_score_
    rmse = np.sqrt(-1*grid_model.best_score_)
    print('{0} After 5 CV, best average MSE: {1}, best average RMSE: {2}, best alpha: {3}'.format(model.__class__.__name__,np.round(mse,4),
                                                                           np.round(rmse,4),grid_model.best_params_))
    print('\n')     

    return grid_model.best_params_

    

ridge_params = {'alpha':[0.05, 0.1, 1, 5, 8, 10, 12, 15, 20]}
lasso_params = {'alpha':[0.001, 0.005, 0.008, 0.05, 0.03, 0.1, 0.5, 1, 5, 10]}
print_best_params(ridge_reg,ridge_params)
print_best_params(lasso_reg,lasso_params)

In [ ]:
lr_reg = LinearRegression()
lr_reg.fit(X_train, y_train)

ridge_reg = Ridge(alpha=20)
ridge_reg.fit(X_train, y_train)

lasso_reg = Lasso(alpha=0.001)
lasso_reg.fit(X_train,y_train)

models = [lr_reg, ridge_reg, lasso_reg]
get_rmses(models)

visualize_coefficient(models)

In [ ]:
# Finding Skewed Feature
from scipy.stats import skew

feature_index = df.dtypes[df.dtypes != 'object'].index

skew_features = df[feature_index].apply(lambda x: skew(x))

skew_feature_top = skew_features[skew_features > 1]
print(skew_feature_top.sort_values(ascending=False))

In [ ]:
# log transformation for skewed values
for feature in skew_feature_top.index:
    nonzero_indices = df[feature] != 0  # 0이 아닌 값의 인덱스를 찾음
    addup = min(df[nonzero_indices][feature])
    df[feature] = np.log1p(df[feature]+addup)

In [ ]:
y_target = df['turn_over_rate']
x_data = df.drop(['company_name','turn_over_rate'], axis = 1, inplace = False)

X_train, X_test, y_train, y_test = train_test_split(x_data, y_target, test_size = 0.1, random_state = 7)

ridge_params = {'alpha':[0.05, 0.1, 1, 5, 8, 10, 12, 15, 20]}
lasso_params = {'alpha':[0.001, 0.005, 0.008, 0.05, 0.03, 0.1, 0.5, 1, 5, 10]}
print_best_params(ridge_reg,ridge_params)
print_best_params(lasso_reg,lasso_params)

In [ ]:
lr_reg = LinearRegression()
lr_reg.fit(X_train, y_train)

ridge_reg = Ridge(alpha=0.05)
ridge_reg.fit(X_train, y_train)

lasso_reg = Lasso(alpha=0.001)
lasso_reg.fit(X_train,y_train)

models = [lr_reg, ridge_reg, lasso_reg]
get_rmses(models)
print('\n')
get_maes(models)

visualize_coefficient(models)

Decision Tree with Regression

xgbooost

In [ ]:
from xgboost import XGBRegressor 

xgb_params = {'n_estimators': [500, 750, 1000], 
              'learning_rate': [0.05, 0.1, 0.15, 0.2], 
              'max_depth' : [4,6,8,10]}
xgb_reg = XGBRegressor(n_estimators = 1000, learning_rate = 0.05, colsample_bytree = 0.5, subsample = 0.8)

xgb_best_params = save_best_params(xgb_reg, xgb_params)

print(xgb_best_params)

In [ ]:
# work after best params
xgb_reg = XGBRegressor(n_estimators = 500, learning_rate = 0.05, max_depth = 4, colsample_bytree = 0.5, subsample = 0.8)
xgb_reg.fit(X_train.astype(float),y_train.astype(float))

get_mae(xgb_reg)
get_rmse(xgb_reg)

light gbm

In [ ]:
from lightgbm import LGBMRegressor

lgbm_params = {'n_estimators': [500, 750, 1000], 
              'learning_rate': [0.05, 0.1, 0.15, 0.2], 
              'num_leaves' : [4, 6, 8, 10]}
lgbm_reg = LGBMRegressor(n_estimators=1000, learning_rate= 0.05, num_leaves=4,
                         subsample=0.6, colsample_bytree=0.4, reg_lambda=10, n_jobs=-1)
lgbm_best_params = save_best_params(lgbm_reg,lgbm_params)

print(lgbm_best_params)

In [ ]:
# work after best params
lgbm_reg = LGBMRegressor(n_estimators=500, learning_rate= 0.05, num_leaves=4,
                         subsample=0.6, colsample_bytree=0.4, reg_lambda=10, n_jobs=-1)
lgbm_reg.fit(X_train.astype(float),y_train.astype(float))

get_mae(lgbm_reg)
get_rmse(lgbm_reg)

gradient boosting

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

gb_params = {'n_estimators': [500, 750, 1000], 
              'learning_rate': [0.05, 0.1, 0.15, 0.2], 
              'max_depth' : [4,6,8,10]}
gb_reg = GradientBoostingRegressor(n_estimators=1000, learning_rate= 0.05, subsample=0.6 )
gb_best_params = save_best_params(gb_reg,gb_params)

print(gb_best_params)

In [ ]:
# work after best params
gb_reg = GradientBoostingRegressor(n_estimators=750, learning_rate= 0.05, subsample=0.6, max_depth=10 )
gb_reg.fit(X_train.astype(float),y_train.astype(float))

get_mae(gb_reg)
get_rmse(gb_reg)

random forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf_params = {'n_estimators': [500, 750, 1000],
             'max_depth' : [4,6,8,10]}
rf_reg = RandomForestRegressor(n_estimators=1000)
rf_best_params = save_best_params(rf_reg,rf_params)

print(rf_best_params)

In [ ]:
# work after best params

rf_reg = RandomForestRegressor(n_estimators=500)
rf_reg.fit(X_train,y_train)

get_mae(rf_reg)
get_rmse(rf_reg)

In [ ]:
from sklearn.tree import DecisionTreeRegressor

dt_params = {'max_depth' : [4,6,8,10]}
dt_reg = DecisionTreeRegressor(max_depth=4)
dt_best_params = save_best_params(dt_reg, dt_params)

print(dt_best_params)

In [ ]:
# work after best params

dt_reg = DecisionTreeRegressor(max_depth=4)
dt_reg.fit(X_train,y_train)

get_mae(dt_reg)
get_rmse(dt_reg)

In [ ]:
# Feature graph xgb
xgb_reg.fit(x_data.astype(float), y_target.astype(float))

feature_series = pd.Series(data = xgb_reg.feature_importances_, index =x_data.columns)
feature_series = feature_series.sort_values(ascending=False)
sns.barplot(x=feature_series, y= feature_series.index)

In [ ]:
# Feature Graph lgbm
lgbm_reg.fit(x_data.astype('float'), y_target.astype('float'))

feature_series = pd.Series(data = lgbm_reg.feature_importances_, index =x_data.columns)
feature_series = feature_series.sort_values(ascending=False)
sns.barplot(x=feature_series, y= feature_series.index)


In [ ]:
# Feature graph gradient boosting

gb_reg.fit(x_data.astype('float'), y_target.astype('float'))

feature_series = pd.Series(data = gb_reg.feature_importances_, index =x_data.columns)
feature_series = feature_series.sort_values(ascending=False)
sns.barplot(x=feature_series, y= feature_series.index)

In [ ]:
# Feature graph randomforest

rf_reg.fit(x_data.astype('float'), y_target.astype('float'))

feature_series = pd.Series(data = rf_reg.feature_importances_, index =x_data.columns)
feature_series = feature_series.sort_values(ascending=False)
sns.barplot(x=feature_series, y= feature_series.index)


In [ ]:
# Feature Graph DecisionTree
dt_reg.fit(x_data.astype('float'), y_target.astype('float'))

feature_series = pd.Series(data = dt_reg.feature_importances_, index =x_data.columns)
feature_series = feature_series.sort_values(ascending=False)
sns.barplot(x=feature_series, y= feature_series.index)

Export Trained Model

In [ ]:
import pickle
import joblib

# joblib.dump(lgbm_reg,"tp_lgbm_reg_model.pkl")
joblib.dump(xgb_reg,"tp_xgb_reg_model.pkl")